In [17]:
%matplotlib ipympl
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
from sklearn.cluster import KMeans
import tabulate
import sys
import os
from scipy import stats
from autorank import autorank,create_report,plot_stats
from significance_analysis import conduct_analysis
pd.set_option("chained_assignment", None)
pd.set_option("display.max_rows", 5000)
pd.set_option("display.max_columns", 5000)
pd.set_option("display.width", 10000)

def load_priorband_data():
    df = pd.read_parquet("data_for_anton.parquet")
    df=df.reset_index()
    df_collection=[]
    for seed_nr in range(50):
        partial_df=df[["benchmark","prior","algorithm","used_fidelity"]]
        partial_df["value"]=df[f"seed-{seed_nr}"]
        partial_df["seed"]=seed_nr
        df_collection.append(partial_df)
        print(f"⚙️ Seed {seed_nr+1}/50", end="\r", flush=True)
    print("✅ Loading data done")
    return pd.concat(df_collection,ignore_index=True)

def add_regret(df:pd.DataFrame,normalize:False):
    best={}
    ranges={}
    print("⚙️ Preparing regret", end="\r", flush=True)
    for benchmark in df["benchmark"].unique():
        best[benchmark]=min(df.loc[df["benchmark"]==benchmark]["value"])
        ranges[benchmark]=max(df.loc[df["benchmark"]==benchmark]["value"])-best[benchmark]
        
    def calculate_simple_regret(row,normalize:bool=False):
        if normalize:
            return abs(best[row["benchmark"]]-row["value"])/ranges[row["benchmark"]]
        return abs(best[row["benchmark"]]-row["value"])
    
    if normalize:
        print("⚙️ Adding regret       ", end="\r", flush=True)
        df['regret'] = df.apply(calculate_simple_regret, axis=1,normalize=True)
        print("✅ Adding regret done                      ")
    else:
        print("⚙️ Adding normalized regret       ", end="\r", flush=True)
        df['norm_regret'] = df.apply(calculate_simple_regret, axis=1,normalize=False)
        print("✅ Adding normalized regret done                      ")
    return df

def create_df_dict(df:pd.DataFrame,experiment:dict):
    df_dict={}
    for p_n,prior_key in enumerate(experiment["priors"]):
        df_dict[prior_key]={}
        for b_n,benchmark in enumerate(experiment["benchmarks"]):
            print(f"⚙️ Prior {prior_key} ({p_n+1}), Benchmark {b_n+1}/{len(experiment['benchmarks'])}      ", end="\r", flush=True)
            df_dict[prior_key][benchmark]={}
            for algo_key,algo_value in experiment["algo_dict"].items():
                df_dict[prior_key][benchmark][algo_key]=df.loc[(df["prior"].isin(experiment["prior_dict"][prior_key]))&(df["algorithm"].isin(algo_value))&(df["benchmark"]==benchmark)].reset_index(drop=True)
    print("✅ Creating df-dict done                     ")
    return df_dict

def create_lmem_incumbent(df_dict:pd.DataFrame,experiment:dict,cumulative:bool=False,value:list[str]=["value"]):
    fs=experiment["f_space"]
    algorithms=experiment["algorithms"]
    algo_group=experiment["algo_group"]
    benchmarks=experiment["benchmarks"]
    priors=experiment["priors"]
    
    output={}
    output["dataset"]={}
    for p_n,prior in enumerate(priors):
        output["dataset"][prior]={}
        for n_f,max_f in enumerate(fs):
            output["dataset"][prior][max_f]={}
            output["dataset"][prior][max_f]=pd.DataFrame()
            for b_n,benchmark in enumerate(benchmarks):
                df_at_point=df_dict[prior][benchmark][algo_group]
                df_at_point=df_at_point.loc[df_at_point["used_fidelity"]<=max_f]
                for seed in df_at_point["seed"].unique():
                    print(f"⚙️ Prior {prior} ({p_n+1}), Fidelity {n_f+1}/{len(fs)}, Benchmark {b_n+1}/{len(benchmarks)}          ", end="\r", flush=True)
                    for algo in algorithms:
                        df_criteria=df_at_point.loc[(df_at_point["seed"]==seed)&(df_at_point["algorithm"]==algo)].iloc[-1].to_frame().T
                        df_criteria["used_fidelity"]=max_f
                        output["dataset"][prior][max_f]=pd.concat([output["dataset"][prior][max_f],df_criteria],ignore_index=True)
            output["dataset"][prior][max_f][value+["used_fidelity"]]=output["dataset"][prior][max_f][value+["used_fidelity"]].astype(float)
            output["dataset"][prior][max_f]["seed"]=output["dataset"][prior][max_f]["seed"].astype(int)
    print("✅ Creating incumbent data done                                              ")
    if not cumulative:
        return output["dataset"]
    cumulative_data={}
    for p_n,prior in enumerate(priors):
        cumulative_data[prior]={}
        for max_f in experiment["budgets"]:
            cumulative_data[prior][max_f]=pd.DataFrame()
            for fidelity in [item for item in experiment["f_space"] if experiment["min_f"] <= item <= max_f]:
                cumulative_data[prior][max_f]=pd.concat([cumulative_data[prior][max_f],output["dataset"][prior][fidelity]],ignore_index=True)

    return cumulative_data

def run_lmems(experiment:dict,values:list[str]=["value"]):
    result_dict={}
    for v_n,value in enumerate(values):
        result_dict[value]={}
        for p_n,priortype in enumerate(experiment["priors"]):
            result_dict[value][priortype]={}
            for f_n,max_f in enumerate(experiment["budgets"]):
                print(f"⚙️ Value {value}({v_n+1}), Prior {priortype}({p_n+1}), Fidelity {f_n+1}/{len(experiment['budgets'])}        ", end="\r", flush=True)
                experiment["dataset"][priortype][max_f]=experiment["dataset"][priortype][max_f].replace({"algorithm": experiment["label_dict"],"benchmark": experiment["label_dict"]})
                real_stdout = sys.stdout
                sys.stdout = open(os.devnull, "w")
                if len(experiment["dataset"][priortype][max_f]["used_fidelity"].unique())>1:
                    print(experiment["dataset"][priortype][max_f]["used_fidelity"].unique())
                    result_dict[value][priortype][max_f]=conduct_analysis(experiment["dataset"][priortype][max_f],value,"algorithm","benchmark",bin_id="used_fidelity",continuous_fidelity=False,show_plots=False,verbosity=0)[1]
                else:
                    result_dict[value][priortype][max_f]=conduct_analysis(experiment["dataset"][priortype][max_f],value,"algorithm","benchmark",continuous_fidelity=False,show_plots=False,verbosity=0)[1]
                sys.stdout = real_stdout
    print("✅ Running experiments done                                                             ")
    return result_dict

def plot_cd_diagrams(experiment:dict,value:str="value"):
    color_dict={"random_search":"red","hyperband":"green","pb_mutation_dynamic_geometric-default-at-target":"blue","priorband":"blue","RS":"red","HB":"green","PB":"blue"}

    _ = plt.figure(figsize=(10,10*len(experiment["budgets"])/3))
    gs=GridSpec(nrows=len(experiment["budgets"]),ncols=len(experiment["priors"]))
    for p_n,prior in enumerate(experiment["priors"]):
        for f_n,fidelity in enumerate(experiment["budgets"]):
            if fidelity>experiment["min_f"] and experiment["granularity"]:
                scoreframe=experiment["result_dict"][value][prior][fidelity][0].loc[experiment["result_dict"][value][prior][fidelity][0]["used_fidelity_bins"]==str(float(fidelity))].sort_values(by=['Estimate'])[["algorithm","Estimate"]]
                contrastframe=experiment["result_dict"][value][prior][fidelity][1].loc[experiment["result_dict"][value][prior][fidelity][1]["used_fidelity_bins"]==str(float(fidelity))][["Sig","algorithm_1","algorithm_2"]]
            else:
                scoreframe=experiment["result_dict"][value][prior][fidelity][0].sort_values(by=['Estimate'])[["algorithm","Estimate"]]
                contrastframe=experiment["result_dict"][value][prior][fidelity][1][["Sig","algorithm_1","algorithm_2"]]

            min_score=scoreframe["Estimate"][0]
            max_score=scoreframe["Estimate"][len(scoreframe["Estimate"])-1]

            significance_lines=[]
            for n_best_algo,best_algo in enumerate(scoreframe["algorithm"]):
                for n_worst_algo,worst_algo in reversed(list(enumerate(scoreframe["algorithm"][n_best_algo+1:]))):
                    significance=contrastframe.loc[((contrastframe["algorithm_1"]==best_algo)&(contrastframe["algorithm_2"]==worst_algo))|((contrastframe["algorithm_2"]==best_algo)&(contrastframe["algorithm_1"]==worst_algo))]["Sig"][0]
                    if significance in ["","."]:
                        new_line=[n_best_algo,n_best_algo+n_worst_algo+1]
                        if not any(existing_pair[0] <= new_line[0] <= existing_pair[1] and existing_pair[0] <= new_line[1] <= existing_pair[1] for existing_pair in significance_lines):
                            significance_lines.append([n_best_algo,n_best_algo+n_worst_algo+1])
                            break
            n_sign_lines=len(significance_lines)
            plot_height=10+5

            ax=plt.subplot(gs[f_n,p_n])
            
            ax.set_title(f"{fidelity}x {prior}",pad=0,y=-0.18)
            ax.get_yaxis().set_visible(False)
            ax.set_xlim(min_score-(max_score-min_score)*0.1,max_score+(max_score-min_score)*0.1)
            ax.set_ylim(0,plot_height)
            ax.spines['bottom'].set_position(('data', 0.0))
            ax.spines[['top',"left","right"]].set_visible(False)
            ax.invert_xaxis()

            for algo in range(len(scoreframe["Estimate"])):
                ax.plot([scoreframe["Estimate"][algo],scoreframe["Estimate"][algo]], [0, plot_height-6], '-', lw=2, label="_not in legend",color=color_dict[scoreframe["algorithm"][algo]])
                ax.text(scoreframe["Estimate"][algo], plot_height-3, scoreframe["algorithm"][algo], horizontalalignment='center',verticalalignment='bottom')
                ax.text(scoreframe["Estimate"][algo], plot_height-5, np.round(scoreframe["Estimate"][algo],2), horizontalalignment='center',verticalalignment='bottom')
            for n_line,line in enumerate(significance_lines):
                if scoreframe["Estimate"][line[0]]==scoreframe["Estimate"][line[1]]:
                    ax.plot([scoreframe["Estimate"][line[0]], scoreframe["Estimate"][line[1]]],[10/(n_sign_lines+1)*(n_line+1)-0.5,10/(n_sign_lines+1)*(n_line+1)+0.5],  '-', lw=2, label="_not in legend",color="gray")
                else:
                    ax.plot([scoreframe["Estimate"][line[0]], scoreframe["Estimate"][line[1]]],[10/(n_sign_lines+1)*(n_line+1),10/(n_sign_lines+1)*(n_line+1)],  '-', lw=6, label="_not in legend",color="gray")
    print(f"Granularity: {experiment['granularity']}, Value: {value}")
    plt.show()


In [2]:
def run_experiment(priors:list[str]=["good+bad"],budgets:list[float]=[1.0,5.0,12.0],granularity:int=0,value:list[str]=["value"],min_f:float=1.0,plot:bool=True,benchmarks:list[str]=["jahs_cifar10","jahs_colorectal_histology","jahs_fashion_mnist","lcbench-126026","lcbench-167190","lcbench-168330","lcbench-168910","lcbench-189906","cifar100_wideresnet_2048","imagenet_resnet_512","lm1b_transformer_2048","translatewmt_xformer_64"]):
    experiment={}
    experiment["algo_dict"]={"rs_hb_pb":["random_search","hyperband","pb_mutation_dynamic_geometric-default-at-target"]}
    experiment["prior_dict"]={"good+bad":["at25","bad"],"good":["at25"],"bad":["bad"]}
    experiment["benchmarks"]=benchmarks
    experiment["label_dict"]={'random_search': 'RS', 'hyperband': 'HB', 'pb_mutation_dynamic_geometric-default-at-target': 'PB', 'jahs_cifar10': 'JAHS-C10', 'jahs_colorectal_histology': 'JAHS-CH', 'jahs_fashion_mnist': 'JAHS-FM', 'lcbench-126026': 'LC-126026', 'lcbench-167190': 'LC-167190', 'lcbench-168330': 'LC-168330', 'lcbench-168910': 'LC-168910', 'lcbench-189906': 'LC-189906', 'cifar100_wideresnet_2048': 'PD1-Cifar100', 'imagenet_resnet_512': 'PD1-ImageNet', 'lm1b_transformer_2048': 'PD1-LM1B', 'translatewmt_xformer_64': 'PD1-WMT',"random_search_prior":"RS+Prior","bo":"BO","pibo":"PiBO","bohb":"BOHB","priorband_bo":"PriorBand+BO"}
    experiment["algo_group"]="rs_hb_pb"
    experiment["algorithms"]=experiment["algo_dict"][experiment["algo_group"]]
    experiment["priors"]=priors
    experiment["budgets"]=budgets
    experiment["min_f"]=min_f
    experiment["granularity"]=granularity
    if experiment["granularity"]:
        experiment["f_space"]=np.round(np.linspace(experiment["min_f"],max(experiment["budgets"]),(experiment["granularity"]*round(max(experiment["budgets"])-experiment["min_f"]))+1),3).tolist()
    else:
        experiment["f_space"]=experiment["budgets"]

    df=load_priorband_data()
    if "regret" in value:
        df=add_regret(df,False)
    if "norm_regret" in value:
        df=add_regret(df,True)
    df_dict=create_df_dict(df,experiment)
    experiment["dataset"]=create_lmem_incumbent(df_dict,experiment,cumulative=experiment["granularity"],value=value)
    experiment["result_dict"]=run_lmems(experiment,value)
    if plot:
        for v in value:
            plot_cd_diagrams(experiment,v)
    return experiment

In [3]:
exp_benchmarks_results={}
for b_n,benchmark in enumerate(["jahs_cifar10","jahs_colorectal_histology","jahs_fashion_mnist","lcbench-126026","lcbench-167190","lcbench-168330","lcbench-168910","lcbench-189906","cifar100_wideresnet_2048","imagenet_resnet_512","lm1b_transformer_2048","translatewmt_xformer_64"]):
    print(f"❗{b_n+1}. Running Benchmark {benchmark}")
    exp_benchmarks_results[benchmark]=run_experiment(["good+bad"],[1,5,12],0,["value","regret","norm_regret"],plot=False,benchmarks=[benchmark])["result_dict"]

❗1. Running Benchmark jahs_cifar10
✅ Loading data done
✅ Adding normalized regret done                      
✅ Adding regret done                      
✅ Creating df-dict done                     
✅ Creating incumbent data done                                              
✅ Running experiments done                                                             
❗2. Running Benchmark jahs_colorectal_histology
✅ Loading data done
✅ Adding normalized regret done                      
✅ Adding regret done                      
✅ Creating df-dict done                     
✅ Creating incumbent data done                                              
✅ Running experiments done                                                             
❗3. Running Benchmark jahs_fashion_mnist
✅ Loading data done
✅ Adding normalized regret done                      
✅ Adding regret done                      
✅ Creating df-dict done                     
✅ Creating incumbent data done                              

In [54]:
dataset=pd.DataFrame(columns=["value_type","benchmark","prior","fidelity","RS - HB","RS - PB","HB - PB"])
for benchmark in exp_benchmarks_results.keys():
    for value in exp_benchmarks_results[benchmark].keys():
        for fidelity in exp_benchmarks_results[benchmark][value]["good+bad"].keys():
            contrastframe=exp_benchmarks_results[benchmark][value]["good+bad"][fidelity][1]
            new_row=[value,benchmark,"good+bad",fidelity,(1-contrastframe["P-val"][0])*np.sign(contrastframe["Estimate"][0]),(1-contrastframe["P-val"][1])*np.sign(contrastframe["Estimate"][1]),(1-contrastframe["P-val"][2])*np.sign(contrastframe["Estimate"][2])]
            new_row=[new_row]
            dataset=pd.concat([dataset,pd.DataFrame(new_row,columns=dataset.columns)],ignore_index=True)
print(dataset)


      value_type                  benchmark     prior fidelity  RS - HB  RS - PB  HB - PB
0          value               jahs_cifar10  good+bad        1   -1.000   -1.000   -1.000
1          value               jahs_cifar10  good+bad        5    0.404   -0.310   -0.825
2          value               jahs_cifar10  good+bad       12    0.917    0.920    0.000
3         regret               jahs_cifar10  good+bad        1   -1.000   -1.000   -1.000
4         regret               jahs_cifar10  good+bad        5    0.404   -0.310   -0.825
5         regret               jahs_cifar10  good+bad       12    0.917    0.920    0.000
6    norm_regret               jahs_cifar10  good+bad        1   -1.000   -1.000   -1.000
7    norm_regret               jahs_cifar10  good+bad        5    0.404   -0.310   -0.825
8    norm_regret               jahs_cifar10  good+bad       12    0.917    0.920    0.000
9          value  jahs_colorectal_histology  good+bad        1   -1.000   -1.000   -1.000
10        

In [94]:
def simple_clustering(k,data_frame, type_column, numerical_columns, value,fidelity,plot:bool=False):
    # Combine data from all categories
    combined_data = pd.DataFrame()
    for type_value in data_frame[type_column].unique():
        type_data = data_frame[data_frame[type_column] == type_value]
        combined_data = pd.concat([combined_data,type_data])

    # Perform clustering on the combined data
    kmeans = KMeans(n_clusters=k,n_init="auto")
    kmeans.fit(combined_data[numerical_columns])
    cluster_labels = kmeans.labels_
    

    # Assign cluster labels to the original data frame
    data_frame['cluster'] = cluster_labels
    def add_cluster_centers(row,columns,means):
        return means[row["cluster"]]

    data_frame[numerical_columns] = data_frame.apply(add_cluster_centers, axis=1,columns=numerical_columns,means=kmeans.cluster_centers_).to_list()

    if plot:
        # Visualize clusters using Matplotlib
        fig=plt.figure()
        ax=fig.add_subplot(projection="3d")
        
        ax.scatter(combined_data[numerical_columns[0]], combined_data[numerical_columns[1]],combined_data[numerical_columns[2]], c=cluster_labels)
        
        for _,row in combined_data.iterrows():
            ax.text(row[numerical_columns].values[0],row[numerical_columns].values[1],row[numerical_columns].values[2],row[type_column],fontsize=5)

        
        plt.title(f"{k} Clusters in {value} and fidelity {fidelity}")
        # ax.set_xlim([-1, 1])
        # ax.set_ylim([-1, 1])
        # ax.set_zlim([-1,1])
        ax.set_xlabel(numerical_columns[0])
        ax.set_ylabel(numerical_columns[1])
        ax.set_zlabel(numerical_columns[2])
        plt.show()

    return data_frame

meta_feature_df=pd.read_csv("benchmark_metafeatures.csv")
def add_meta_features(row):
    return meta_feature_df.loc[meta_feature_df["code_name"]==row["benchmark"]][["# Vars","# cont. Vars","# cond. Vars","# cat. Vars","log int","int"]].values[0]

clusters={}
for value in dataset["value_type"].unique():
    clusters[value]={}
    for fidelity in dataset["fidelity"].unique():
        sample_df=dataset.loc[(dataset["value_type"]==value)&(dataset["fidelity"]==fidelity)][["benchmark","RS - HB","HB - PB","RS - PB"]]
        clusters[value][fidelity]=simple_clustering(3,sample_df,"benchmark",["RS - HB","HB - PB","RS - PB"],value,fidelity,plot=False)
        clusters[value][fidelity][["# Vars","# cont. Vars","# cond. Vars","# cat. Vars","log int","int"]]=clusters[value][fidelity].apply(add_meta_features,axis=1).to_list()
        print(f"Dataframe for value-type {value} at {fidelity}x Budget \n",clusters[value][fidelity],"\n")


Dataframe for value-type value at 1x Budget 
                     benchmark   RS - HB  HB - PB  RS - PB  cluster  # Vars  # cont. Vars  # cond. Vars  # cat. Vars  log int  int
0                jahs_cifar10 -1.000000   -1.000     -1.0        0      14             2             0            8        0    1
9   jahs_colorectal_histology -1.000000   -1.000     -1.0        0      14             2             0            8        0    1
18         jahs_fashion_mnist -1.000000   -1.000     -1.0        0      14             2             0            8        0    1
27             lcbench-126026  0.998857    0.718      1.0        1       7             4             0            0        0    1
36             lcbench-167190  0.998857    0.718      1.0        1       7             4             0            0        0    1
45             lcbench-168330  0.998857    0.718      1.0        1       7             4             0            0        0    1
54             lcbench-168910  0.998857    0